# Комбинаторный аукцион

Задача решена с помощью генетического алгоритма (код представлен ниже).

Генетический алгоритм не гарантирует, что найденное решение будет оптимальным (т.е. суммарная стоимость будет максимально возможной).

Чтобы повысить вероятность нахождения оптимального решения применён алгоритм имитации отжига и проведено несколько запусков кода с различными значениями гиперпараметра алгоритма (genetic_line_max_age = максимальный возраст генетической линии).

Получены следующие результаты:

<br>
<br>

**Часть 1.** Суммарная стоимость = 643.

|Номер игрока|Товары|Стоимость
|---|---|---|
|1|C|91|
|2|H|88|
|3|I|87|
|4|AD|69|
|5|E|67|
|6|BG|72|
|7|J|90|
|8|F|79|

<br>
<br>

**Часть 2. Пункт а.** Суммарная стоимость = 3757.

|Номер игрока|Товары|Стоимость
|---|---|---|
|1|NP|450|
|2|ACEFGJKLMR|2196|
|3|BI|442|
|4|H|261|
|5|Пусто|0|
|6|O|201|
|7|D|207|

<br>
<br>

**Часть 2. Пункт б.** Суммарная стоимость = 4036.

|Номер игрока|Товары|Стоимость
|---|---|---|
|1|Пусто|0|
|2|EFJK|930|
|3|BI|438|
|4|CDGHLMNOPRS|2530|
|5|A|138|
|6|Пусто|0|
|7|Пусто|0|


In [1]:
import random
import numpy as np
import pandas as pd
from bisect import bisect_left  # для алгоритма имитации отжига
from math import exp  # для алгоритма имитации отжига
from datetime import datetime

## Реализация генетического алгоритма (GeneticMachinery) в общем случае

In [2]:
# класс Хромосомы (содержит геном и значение фитнес-функции этого генома)
class Chromosome:
    genome = None
    fitness = None
    age = 0

    def __init__(
        self,
        genome,
        fitness,
    ) -> None:
        self.genome = genome
        self.fitness = fitness

    def __str__(self):
        return f' Genome: {self.genome}.\n Fitness: {self.fitness}.'


# реализация генетического алгоритма в общем случае
class GeneticMachinery:

    def __init__(
        self,
        available_genes_set: list,  # список возможных генов
        get_fitness,  # фитнес-функция
        genome_length: int,  # длина генома
        optimal_fitness,  # оптимальное значение фитнес-функции (если неизвестно, то ставим значение, которое хотим достичь)
        printing,  # функция печати процесса эволюции
    ) -> None:
        self.available_genes_set = available_genes_set
        self.get_fitness = get_fitness
        self.genome_length = genome_length
        self.optimal_fitness = optimal_fitness
        self.printing = printing
    
    # генерирование хромосомы родителя
    def _generate_parent(
        self,
    ) -> Chromosome:
        genome = []
        while len(genome) < self.genome_length:
            # заполняем геном родителя рандомными генами
            genome.append(random.choice(self.available_genes_set))
        # считаем значение фитнес-функции полученного генома
        fitness = self.get_fitness(genome)
        # возвращаем хромосому родителя
        return Chromosome(genome=genome, fitness=fitness)
    
    # генерирование хромосомы ребёнка
    def _mutate(
        self,
        parent: Chromosome,  # хромосома родителя
    ) -> Chromosome:
        # заменяем рандомный ген в геноме другим доступным геном
        index = random.randrange(0, len(parent.genome))
        child_genome = parent.genome[:]
        new_gene, alternate = random.sample(self.available_genes_set, 2)
        child_genome[index] = alternate if new_gene == child_genome[index] else new_gene
        # и ещё один ген в рандомных случаях
        if random.randint(0,1):
            index = random.randrange(0, len(parent.genome))
            new_gene, alternate = random.sample(self.available_genes_set, 2)
            child_genome[index] = alternate if new_gene == child_genome[index] else new_gene
        # считаем значение фитнес-функции полученного генома
        fitness = self.get_fitness(child_genome)
        # возвращаем хромосому ребёнка
        return Chromosome(genome=child_genome, fitness=fitness)
    
    def _get_improvement(
            self,
            genetic_line_max_age,
            view_evolution,
            start_time
    ):
        parent = self._generate_parent()
        best_parent = Chromosome(genome=parent.genome.copy(), fitness=parent.fitness)
        yield best_parent
        historical_fitnesses = [best_parent.fitness]
        while True:
            child = self._mutate(parent)
            if parent.fitness > child.fitness:
                if genetic_line_max_age is None:
                    continue
                parent.age += 1
                if genetic_line_max_age > parent.age:
                    continue
                child_fitness_index = bisect_left(historical_fitnesses, child.fitness, 0, len(historical_fitnesses))
                best_and_child_diff = len(historical_fitnesses) - child_fitness_index
                similar_prop = best_and_child_diff / len(historical_fitnesses)
                if random.random() < exp(-similar_prop):
                    if view_evolution:
                        print('New Genetic Line')
                    #parent = child
                    parent = self._generate_parent()
                    continue
                parent = best_parent
                parent.age = 0
                continue
            if not child.fitness > parent.fitness:
                child.age = parent.age + 1
                parent = child
                continue
            if view_evolution:
                self.printing(current_generation=child, start_time=start_time)
            parent = child
            parent.age = 0
            if child.fitness > best_parent.fitness:
                yield child
                best_parent = child
                historical_fitnesses.append(child.fitness)
    
    # основной метод генетического алгоритма (выбор лучшей хромосомы)
    def get_the_best(
        self,
        genetic_line_max_age=None,
        view_evolution=False
    ) -> Chromosome:
        # фуксируем random seed для повторяемости результатов
        #random.seed(42)
        # начинаем отсчёт времени
        start_time = datetime.now()
        for improvement in self._get_improvement(genetic_line_max_age=genetic_line_max_age,
                                                 view_evolution=view_evolution,
                                                 start_time=start_time):
            if not self.optimal_fitness > improvement.fitness:
                return improvement

## Реализация генетического алгоритма для задачи комбинаторного аукциона (в виде надстройки к реализации в общем случае)

В качестве генома выступает список длиной $n$ с возможными значениями элементов от 1 до $k$
(где $n$ - число предметов аукциона, а $k$ - число игроков (покупателей), участвующих в аукционе)

Каждый элемент списка означает номер покупателя, который купит соответствующий (позиция в списке) предмет.

**Пример.**

Пусть genome = [1, 2, 1, 3, 4, 5, 6, 7, 8, 8].

Тогда:

покупатель 1 (элемент списка) купит предметы 1 (A) и 3 (C) (((позиции элемента в списке)));

покупатель 2 купит предмет 2 (B);

покупатель 3 купит предмет 4 (D);

покупатель 4 купит предмет 5 (E);

покупатель 5 купит предмет 6 (F);

покупатель 6 купит предмет 7 (G);

покупатель 7 купит предмет 8 (H);

покупатель 8 купит предметы 9 (I) и 10 (J).

In [3]:
# фитнес-функция для задачи комбинаторного аукциона (полученная суммарная стоимость)
class FitnessForAuctionProblem:
    auction_sum = None

    def __init__(
        self,
        auction_sum: int,
    ) -> None:
        self.auction_sum = auction_sum

    def __gt__(
        self,
        other
    ) -> bool:
        return self.auction_sum > other.auction_sum

    def __str__(
        self
    ) -> str:
        return f'(sum: {self.auction_sum})'


# функция для вычисления фитнес-функции для заданного генома
def get_fitness_for_auction_problem(
    auction_matrix: np.array,
    genome: list
) -> FitnessForAuctionProblem:
    # вычисление полученной суммарной стоимости для данного генома
    auction_sum_for_genome = 0
    for buyer in set(genome):
        if buyer != 0:  # 0 означает, что предмет не достался никому (и прибыли за него нет)
            indices = [i for i, x in enumerate(genome) if x == buyer]
            # по условию задачи ценность для произвольного подмножества есть ценность самого дорогого предмета в подмножестве
            value = max(auction_matrix[buyer-1, indices])  # buyer-1, так как нумерация элементов списка начинается с нуля
            auction_sum_for_genome += value
    # возвращаем фитнес-функцию с зашитым условием сравнения
    return FitnessForAuctionProblem(
        auction_sum=auction_sum_for_genome,
    )

# функция печати текущего состояния эволюционного процесса
def printing_for_auction_problem(
    current_generation: Chromosome,  # текущая хромосома
    start_time: datetime,  # время начала эволюции
) -> None:
    time_diff = datetime.now() - start_time
    print(','.join([str(x) for x in current_generation.genome]) +
          f'\t{current_generation.fitness}\t{time_diff}')


# изменим функцию мутации общей реализации генетического алгоритма
# сделаем её более гибкой - стараемся избежать попадания в локальный экстремум (но не факт, что получится)
# можно экспериментировать с различными алгоритмами мутации, чтобы достичь наиболее хороших результатов
class GeneticMachineryForAuctionProblem(GeneticMachinery):

    def _mutate(
        self,
        parent: Chromosome,
    ) -> Chromosome:
        child_genome = parent.genome[:]
        if len(self.available_genes_set) == len(set(child_genome)):
            count = random.randint(1, 7)
            while count > 0:
                count -= 1
                index_a, index_b = random.sample(range(self.genome_length), 2)
                child_genome[index_a], child_genome[index_b] = child_genome[index_b], child_genome[index_a]
        else:
            index_a = random.randrange(0, self.genome_length)
            index_b = random.randrange(0, len(self.available_genes_set))
            child_genome[index_a] = self.available_genes_set[index_b]
            
        fitness = self.get_fitness(child_genome)
        return Chromosome(genome=child_genome, fitness=fitness)

### 1 часть

In [4]:
# матрица ценности предметов (ставок): строки - участники; столбцы - предметы
auction_matrix_1 = np.array([[88, 89, 91, 87, 92, 86, 89, 80, 90, 93],
                             [54, 69, 58, 68, 74, 85, 88, 88, 89, 91],
                             [73, 78, 79, 70, 76, 81, 75, 78, 87, 88],
                             [64, 61, 60, 69, 68, 79, 74, 70, 56, 89],
                             [59, 64, 61, 64, 67, 80, 73, 68, 62, 88],
                             [52, 50, 58, 49, 50, 77, 72, 67, 57, 87],
                             [58, 57, 59, 56, 58, 76, 70, 66, 45, 90],
                             [49, 48, 57, 47, 49, 79, 71, 67, 50, 86]])

k = len(auction_matrix_1)  # количество игороков (покупателей)
n = len(auction_matrix_1[0])  # количество предметов

# решение задачи комбинаторного аукциона
def main_for_auction_problem_1():
    auction_problem_genetic_machinery = GeneticMachineryForAuctionProblem(
        available_genes_set=[i + 1 for i in range(k)],
        get_fitness=lambda genome: get_fitness_for_auction_problem(auction_matrix=auction_matrix_1, genome=genome),
        genome_length=n,
        # далее ставим целевое зн-ие фитнес-функции (оставил зн-ие, которое удалось достичь;
        # при бОльших зн-ях не удаётся найти геном за разумное время)
        optimal_fitness=FitnessForAuctionProblem(643),
        printing=printing_for_auction_problem
    )
    print(auction_problem_genetic_machinery.get_the_best(genetic_line_max_age=5000, view_evolution=False))

main_for_auction_problem_1()

 Genome: [5, 5, 1, 4, 5, 8, 6, 2, 3, 7].
 Fitness: (sum: 643).


In [5]:
# значение фитнес-функции для найденного генома
# можно поэкспериментировать внося мутации в геном вручную
get_fitness_for_auction_problem(auction_matrix=auction_matrix_1,
                                genome=[4,6,1,4,5,8,6,2,3,7]).auction_sum

643

### 2 часть

In [6]:
# функция для вычисления фитнес-функции для заданного генома
def get_fitness_for_auction_problem_2(
    auction_matrix: pd.DataFrame,
    items_dict: dict,
    genome: list
) -> FitnessForAuctionProblem:
    # вычисление полученной суммарной стоимости для данного генома
    auction_sum_for_genome = 0
    for buyer in set(genome):
        if buyer != 0:  # 0 означает, что предмет не достался никому (и прибыли за него нет)
            indices = [i for i, x in enumerate(genome) if x == buyer]
            items_str = ''
            for i in indices:
                items_str += items_dict[i]
            # по условию задачи ценность для произвольного подмножества есть ценность самого дорогого предмета в подмножестве
            value = auction_matrix.loc[buyer, items_str]
            auction_sum_for_genome += value
    # возвращаем фитнес-функцию с зашитым условием сравнения
    return FitnessForAuctionProblem(
        auction_sum=auction_sum_for_genome,
    )

#### пункт а

In [7]:
auction_matrix_2a = pd.read_csv('data_1.csv', index_col=0, header=None).T
auction_matrix_2a

,id,A,B,C,D,E,F,G,H,I,...,ABCDEFGHJKLMNOPR,ABCDEFGIJKLMNOPR,ABCDEFHIJKLMNOPR,ABCDEGHIJKLMNOPR,ABCDFGHIJKLMNOPR,ABCEFGHIJKLMNOPR,ABDEFGHIJKLMNOPR,ACDEFGHIJKLMNOPR,BCDEFGHIJKLMNOPR,ABCDEFGHIJKLMNOPR
1,1,100,139,178,158,149,148,220,188,179,...,2325,2975,2621,2925,2508,2791,2825,3107,2634,2824
2,2,110,129,179,150,211,218,180,179,178,...,2867,3056,3082,2773,2739,2516,2817,3122,2977,3137
3,3,119,212,138,181,148,210,178,159,227,...,2850,2484,2296,2781,2836,2710,3013,2606,2828,3551
4,4,120,190,208,199,120,137,198,261,170,...,2666,2987,2868,3143,2772,2994,2805,2956,2985,2865
5,5,140,159,149,169,177,159,190,199,189,...,2655,2435,2843,2857,2732,2737,2534,2818,2838,2605
6,6,130,151,188,199,161,169,171,208,179,...,2938,2790,2652,2981,2784,2705,2444,2854,2826,3075
7,7,100,170,169,207,181,160,199,170,200,...,3162,3141,2474,2782,2535,2533,2902,2620,2302,3123


In [8]:
items_dict_2a = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H',
    8: 'I',
    9: 'J',
    10: 'K',
    11: 'L',
    12: 'M',
    13: 'N',
    14: 'O',
    15: 'P',
    16: 'R'
}

k = len(auction_matrix_2a)  # количество игороков (покупателей)
n = len(items_dict_2a)  # количество предметов

# решение задачи комбинаторного аукциона
def main_for_auction_problem_2a():
    auction_problem_genetic_machinery = GeneticMachinery(
        available_genes_set=[i + 1 for i in range(k)],
        get_fitness=lambda genome: get_fitness_for_auction_problem_2(auction_matrix=auction_matrix_2a,
                                                                     items_dict=items_dict_2a,
                                                                     genome=genome),
        genome_length=n,
        # далее ставим целевое зн-ие фитнес-функции (оставил зн-ие, которое удалось достичь;
        # при бОльших зн-ях не удаётся найти геном за разумное время)
        optimal_fitness=FitnessForAuctionProblem(3757),
        printing=printing_for_auction_problem
    )
    print(auction_problem_genetic_machinery.get_the_best(genetic_line_max_age=2000, view_evolution=False))

main_for_auction_problem_2a()

 Genome: [2, 3, 2, 7, 2, 2, 2, 4, 3, 2, 2, 2, 2, 1, 6, 1, 2].
 Fitness: (sum: 3757).


In [9]:
# значение фитнес-функции для найденного генома
# можно поэкспериментировать внося мутации в геном вручную
get_fitness_for_auction_problem_2(auction_matrix=auction_matrix_2a,
                                  items_dict=items_dict_2a,
                                  genome=[2,3,2,7,2,2,2,4,3,2,2,2,2,1,6,1,2]).auction_sum

3757

#### пункт б

In [10]:
auction_matrix_2b = pd.read_csv('data_2.csv', index_col=0, header=None).T
auction_matrix_2b

,id,A,B,C,D,E,F,G,H,I,...,ABCDEFGHJKLMNOPRS,ABCDEFGIJKLMNOPRS,ABCDEFHIJKLMNOPRS,ABCDEGHIJKLMNOPRS,ABCDFGHIJKLMNOPRS,ABCEFGHIJKLMNOPRS,ABDEFGHIJKLMNOPRS,ACDEFGHIJKLMNOPRS,BCDEFGHIJKLMNOPRS,ABCDEFGHIJKLMNOPRS
1,1,100,139,177,161,151,150,220,189,178,...,3090,2804,3241,2901,3519,2724,2522,2659,2950,3609
2,2,110,128,180,150,210,220,181,178,181,...,2756,3468,2705,2695,2846,2713,2384,2874,3388,3383
3,3,119,208,139,181,150,207,180,159,231,...,3075,3029,3051,2812,3406,2806,2404,3355,2599,2383
4,4,119,190,210,198,121,140,199,258,169,...,3177,2955,2924,2754,3151,3224,3288,3327,3387,3638
5,5,138,159,147,168,178,157,190,199,191,...,2967,2853,2824,3046,2580,2888,2589,2910,2935,3500
6,6,130,150,188,198,158,168,169,210,178,...,3045,2936,2773,2924,2533,3116,3037,3118,2984,2661
7,7,100,169,170,208,179,159,200,168,198,...,2959,2651,3068,3305,3135,2722,3161,2968,2873,2773


In [11]:
items_dict_2b = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E',
    5: 'F',
    6: 'G',
    7: 'H',
    8: 'I',
    9: 'J',
    10: 'K',
    11: 'L',
    12: 'M',
    13: 'N',
    14: 'O',
    15: 'P',
    16: 'R',
    17: 'S'
}

k = len(auction_matrix_2b)  # количество игороков (покупателей)
n = len(items_dict_2b)  # количество предметов

# решение задачи комбинаторного аукциона
def main_for_auction_problem_2b():
    auction_problem_genetic_machinery = GeneticMachinery(
        available_genes_set=[i + 1 for i in range(k)],
        get_fitness=lambda genome: get_fitness_for_auction_problem_2(auction_matrix=auction_matrix_2b,
                                                                     items_dict=items_dict_2b,
                                                                     genome=genome),
        genome_length=n,
        # далее ставим целевое зн-ие фитнес-функции (оставил зн-ие, которое удалось достичь;
        # при бОльших зн-ях не удаётся найти геном за разумное время)
        optimal_fitness=FitnessForAuctionProblem(4036),
        printing=printing_for_auction_problem
    )
    print(auction_problem_genetic_machinery.get_the_best(genetic_line_max_age=5000, view_evolution=False))

main_for_auction_problem_2b()

 Genome: [5, 3, 4, 4, 2, 2, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4].
 Fitness: (sum: 4036).


In [12]:
# значение фитнес-функции для найденного генома
# можно поэкспериментировать внося мутации в геном вручную
get_fitness_for_auction_problem_2(auction_matrix=auction_matrix_2b,
                                  items_dict=items_dict_2b,
                                  genome=[5,3,4,4,2,2,4,4,3,2,2,4,4,4,4,4,4,4]).auction_sum

4036